# Задание 3

In [ ]:
from collections import OrderedDict
import functools
import inspect
import datetime
import time
import sys
from functools import lru_cache

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [ ]:
class LruCache(object):
    def __init__(self, max_count):
        self.max_count = max_count
        # в современных версиях и обычный dict отсортированный, но ладно.
        self.cache = OrderedDict()

    def __getitem__(self, key):
        if key not in self.cache:
            return None
        else:
            self.cache[key] = self.cache.pop(key)
            return self.cache[key]

    def __setitem__(self, key, value):
        if key in self.cache:
            del self.cache[key]
        self.cache[key] = value
        if len(self.cache) > self.max_count:
            self.cache.popitem(last=False)

#### Реализуйте декоратор

In [ ]:
def cached(max_count):

    def decorator(func):
        cache = LruCache(max_count)
        # разные аргументы args и kwargs могут на самом деле быть одним и тем же
        signature = inspect.signature(func)
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key_mut = signature.bind(*args, **kwargs).arguments
            key_immut = tuple([(arg, key_mut[arg]) for arg in sorted(key_mut.keys())])

            # перегрузили гетитем
            result = cache[key_immut]
            if result is None:
                result = func(*args, **kwargs)
                cache[key_immut] = result
            
            return result
        
        return wrapper
    
    return decorator

#### Проверьте использование декоратора

In [ ]:
def check_time(func, n):
    start = time.time()
    for i in range(n):
        func(i + 1)
    return time.time() - start

In [ ]:
@cached(20)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

@lru_cache(maxsize=20)
def fact2(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

#### Сравните свою реализацию с lru_cache из functools

In [ ]:
print("С нашим cache", check_time(fact, 600))
print("С lru_cache", check_time(fact2, 600))
print("Без кэша", check_time(fact1, 600))

С нашим cache 0.013468742370605469
С lru_cache 0.04484224319458008
Без кэша 0.04625749588012695


Моя реализация работает неприлично хорошо)

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [ ]:
def checked(*types):

    def decorator(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if len(types) != len(args):
                raise TypeError("Передано не то количество аргументов")

            for arg, tp in zip(args, types):
                if not isinstance(arg, tp):
                    raise TypeError(f"Тип {type(arg)} не соответствует типу {tp}")

            return func(*args, **kwargs)
        
        return wrapper
    
    return decorator

#### Проверьте использование декоратора

In [ ]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [ ]:
strange_func('Егор', 20, ['Москва', 'Долгопрудный'])

In [ ]:
strange_func('Егор', 20, ['Москва', 'Долгопрудный'], 'Пора писать диплом?')

TypeError: ignored

Видимо не пора :с

In [ ]:
strange_func(20, 'Егор', ['Москва', 'Долгопрудный'])

TypeError: ignored

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

В задании просят один логгер для всех функций, применяем паттерн синглтон

In [ ]:
def singleton(cls):
    instance = None

    @functools.wraps(cls)
    def getinstance(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance

    return getinstance

In [ ]:
a = singleton(LruCache)

In [ ]:
a(10)

In [ ]:
a(10)

In [ ]:
a(1)

In [ ]:
@singleton
class Logger:
    def __init__(self):
        self.index = 0
    
    def log(self, file, func_nm, args, kwargs, result):
        log_line = f'{self.index} {datetime.datetime.now()} {func_nm} {args} {kwargs} {result}\n'

        if file in [sys.stdout, sys.stderr]:
            file.write(log_line)
        else:
            with open(file, 'a') as f:
                f.write(log_line)
                
        self.index += 1

In [ ]:
def logger(file=sys.stdout):
    
    logger_ = Logger()
    def decorator(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)

            logger_.log(
                file,
                func.__name__, 
                args, 
                kwargs, 
                result
            )

            return result

        return wrapper

    return decorator

Посмотрим, что все ок

In [ ]:
@logger(sys.stdout)
def test1(*args, **kwargs):
    return 0

@logger(sys.stderr)
def test2(*args, **kwargs):
    return 1

@logger('example_.txt')
def test3(*args, **kwargs):
    return 2

In [ ]:
test1(1, a=2)

4 2021-04-30 19:09:35.663659 test1 (1,) {'a': 2} 0


0

In [ ]:
test2(2, b=3)

5 2021-04-30 19:09:36.003683 test2 (2,) {'b': 3} 1


1

In [ ]:
test3(3, c=4)
with open('example_.txt', 'r') as f:
    print(f.readlines()[-1])

6 2021-04-30 19:09:36.325402 test3 (3,) {'c': 4} 2

